In [1]:
import numpy as np
import pycbc.conversions as pc
import pycbc.pnutils as pnu
import pandas as pd
import h5py, os

from collections import defaultdict

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import random

import sys
from lal import YRJUL_SI

import seaborn as sns

import ast

In [3]:
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

def load_dict_from_hdf5(filename):
    """
    ....
    """
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')

def recursively_load_dict_contents_from_group(h5file, path):
    """
    ....
    """
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item[()]
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
            
    return ans


def print_some_rcParams():
    rc_keys = ['backend', 'axes.labelsize', 'axes.titlesize', 'font.size',
        'legend.fontsize', 'xtick.labelsize', 'ytick.labelsize', 'font.family',
        'text.usetex', 'figure.figsize'] # 'font.sans-serif', 'font.serif'
    print('Using the following matplotlib settings:')
    for k in rc_keys:
        print(k, ':', plt.rcParams[k])

In [4]:
ifar_threshold = 10 ## yr 
number_injection = 10

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [6]:
#This is the first file, all events are true
injection_file = '/home/bhooshan.gadre/work/O3/sub_solar_search/O3B_VTs/ssm-search/O3b_VTs/common_ssm_injections.hdf'
injections = load_dict_from_hdf5(injection_file)
injections.keys(), len(injections.keys())

(dict_keys(['coa_phase', 'declination', 'distance', 'gstlal_ifar', 'gstlal_weight', 'inclination', 'mass1_source', 'mass2_source', 'mbta_ifar', 'polarization', 'pycbc_ifar', 'right_ascension', 'snr_H', 'snr_L', 'snr_V', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z', 'time']),
 22)

In [7]:
df_true = pd.DataFrame(injections)
df_true

,coa_phase,declination,distance,gstlal_ifar,gstlal_weight,inclination,mass1_source,mass2_source,mbta_ifar,polarization,pycbc_ifar,right_ascension,snr_H,snr_L,snr_V,spin1x,spin1y,spin1z,spin2x,spin2y,spin2z,time
0,1.463821,-1.232698,231.483117,0.0,0.666667,2.867627,1.692460,0.501600,0.0,2.515262,0.0,0.164688,2.440571,2.957571,1.361124,0.053696,0.053141,-0.037187,0.057080,-0.011039,0.007150,1.238166e+09
1,0.673478,-0.622410,458.599031,0.0,1.000000,0.918732,6.421466,0.763863,0.0,0.135494,0.0,2.177941,2.684150,2.988815,1.259368,-0.025080,-0.011637,0.021298,0.020269,0.071086,0.038439,1.238166e+09
2,5.447231,-1.440067,302.357273,0.0,0.666667,0.854144,10.481894,1.045793,0.0,0.132165,0.0,2.621765,4.408046,6.731172,1.315282,-0.004171,-0.046270,0.012784,0.021928,-0.027524,-0.030501,1.238166e+09
3,5.521864,-1.276307,463.341251,0.0,0.666667,0.494481,8.906005,0.811978,0.0,1.828451,0.0,3.177169,2.049177,3.539541,0.762279,0.000633,0.001265,-0.000504,0.004547,-0.011592,-0.004068,1.238166e+09
4,0.183151,-0.734805,833.927353,0.0,1.000000,2.895566,7.129897,0.808812,0.0,2.805147,0.0,2.219308,2.389652,3.215351,0.549881,-0.051269,0.011986,-0.001127,-0.021172,0.014852,-0.003302,1.238166e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916578,0.187059,-0.469964,752.284246,0.0,1.000000,0.388063,5.784159,0.559250,0.0,3.122078,0.0,4.757083,2.959844,2.958614,0.812961,-0.032232,0.036856,-0.038265,-0.001476,0.035159,-0.032295,1.269562e+09
1916579,0.141624,-0.272180,643.259480,0.0,0.666667,3.032152,8.621294,0.868851,0.0,0.817515,0.0,1.586071,2.718237,3.586084,3.759479,-0.007036,-0.019932,0.032837,-0.038478,0.020862,-0.010397,1.269562e+09
1916580,1.174031,-0.227858,349.130344,0.0,0.666667,2.206089,10.560706,0.717379,0.0,1.318691,0.0,3.475683,3.402718,2.469075,1.012893,0.004500,-0.054032,-0.029833,0.062644,0.023295,-0.031378,1.269562e+09
1916581,5.752044,0.708430,455.222188,0.0,1.000000,2.115026,5.727628,0.886952,0.0,1.553802,0.0,2.464774,2.854940,3.862371,0.634093,-0.028952,-0.016063,-0.024699,-0.048624,-0.022375,0.007844,1.269562e+09


In [8]:
df_true = df_true.drop(columns=['coa_phase', 'declination', 'distance', 'gstlal_ifar', 'gstlal_weight', 'inclination', 'mbta_ifar', 'polarization', 
                                'right_ascension', 'snr_H', 'snr_L', 'snr_V', 'spin1x', 'spin1y', 'spin2x', 'spin2y', 'time'])

#df_true['label'] = 1

df_true = df_true.rename(columns={'mass2_source': 'mass2', 
                                  'mass1_source': 'mass1'})

df_true

,mass1,mass2,pycbc_ifar,spin1z,spin2z
0,1.692460,0.501600,0.0,-0.037187,0.007150
1,6.421466,0.763863,0.0,0.021298,0.038439
2,10.481894,1.045793,0.0,0.012784,-0.030501
3,8.906005,0.811978,0.0,-0.000504,-0.004068
4,7.129897,0.808812,0.0,-0.001127,-0.003302
...,...,...,...,...,...
1916578,5.784159,0.559250,0.0,-0.038265,-0.032295
1916579,8.621294,0.868851,0.0,0.032837,-0.010397
1916580,10.560706,0.717379,0.0,-0.029833,-0.031378
1916581,5.727628,0.886952,0.0,-0.024699,0.007844


In [9]:
#this the background
trig_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/STATMAP/H1L1V1-COMBINE_STATMAP_FULL_DATA-1238787802-853441.hdf"
bank_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/BANK/H1L1V1-BANK2HDF-1238787802-853441.hdf"

In [10]:
%%time
bank = h5py.File(bank_file, 'r')
hashes = bank['template_hash'][:]

bank_tids = hashes.argsort()
fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
trigs = h5py.File(trig_file, 'r')

trig_params = defaultdict(list)
for tid in trigs['background']['template_id'][:number_injection]:  ## test with 10 triggers. Later this can be [:] for all triggers
    bank_idx = np.argwhere(bank_tids == tid)[0]
    for k in fields:
        trig_params[k].append(bank[k][:][bank_idx][0])
#print(trig_params)

CPU times: user 323 ms, sys: 212 ms, total: 535 ms
Wall time: 580 ms


In [11]:
print(trig_params)

defaultdict(<class 'list'>, {'mass1': [0.5049287468690198, 1.4890962096222466, 0.5584945606105989, 0.30992199092256534, 0.20621452536587256, 6.0120204674999655, 0.9745523320171033, 0.5214335060019926, 0.5423446824343688, 1.0842931303911676], 'mass2': [0.21458005221417903, 0.21205771715862032, 0.2044448030937099, 0.2008728164584661, 0.20621452536587256, 0.7340735619694843, 0.2556743567108287, 0.20865151534506626, 0.20605241989917558, 0.27170560413115535], 'spin1z': [-0.6862350742339347, -0.7286678705958807, 0.7745876481316094, -0.09982390851178215, 0.09989187095218796, 0.8999412988935003, 0.4513576754964428, 0.6390865474789664, -0.7355112205159183, -0.5541470127492194], 'spin2z': [-0.056092396754344645, 0.024225545145718428, 0.006712229612723681, -0.07828719811217069, 0.09990277072233664, 0.41893731416659385, -0.08913105732121088, 0.08529964032606609, 0.010133644484569012, 0.022674100783847937]})


In [12]:
len(trigs['background']['template_id'])

39955275

In [13]:
len(trigs['background_h0']['H1L1']['ifar'])

16322600

In [14]:
len(trigs['background_h0']['H1L1V1']['ifar'][:])

127274

In [15]:
trigs['background_h0']['H1L1V1']['ifar'][:10]

array([1.83540772, 0.02920396, 4.81109905, 0.03547405, 0.07905803,
       2.01532598, 0.00685824, 0.51929064, 0.03567827, 0.38252941])

In [16]:
background_file = '/home/lucashernan.quiceno/background/output.txt'

with open(background_file, 'r') as file:
    content = file.read()

# Convert the contents of the file into a Python dictionary
data_dict = ast.literal_eval(content.replace("defaultdict(<class 'list'>, ", "").rstrip(')'))

# Convert the dictionary to a DataFrame
df_false = pd.DataFrame(data_dict)

df_false

,mass1,mass2,spin1z,spin2z
0,0.504929,0.214580,-0.686235,-0.056092
1,1.489096,0.212058,-0.728668,0.024226
2,0.558495,0.204445,0.774588,0.006712
3,0.309922,0.200873,-0.099824,-0.078287
4,0.206215,0.206215,0.099892,0.099903
...,...,...,...,...
1916578,0.864843,0.247605,-0.416889,0.025385
1916579,0.501223,0.202115,-0.743296,-0.011618
1916580,0.956881,0.213006,-0.813010,0.049815
1916581,0.808391,0.312963,-0.449955,-0.030445


In [17]:
#df_false = pd.DataFrame(trig_params)

df_false['pycbc_ifar'] = trigs['background_h0']['H1L1']['ifar'][:1916583]

df_false['label'] = 0
df_false

,mass1,mass2,spin1z,spin2z,pycbc_ifar,label
0,0.504929,0.214580,-0.686235,-0.056092,0.000314,0
1,1.489096,0.212058,-0.728668,0.024226,0.000070,0
2,0.558495,0.204445,0.774588,0.006712,0.000018,0
3,0.309922,0.200873,-0.099824,-0.078287,0.000821,0
4,0.206215,0.206215,0.099892,0.099903,0.000288,0
...,...,...,...,...,...,...
1916578,0.864843,0.247605,-0.416889,0.025385,0.000010,0
1916579,0.501223,0.202115,-0.743296,-0.011618,0.000011,0
1916580,0.956881,0.213006,-0.813010,0.049815,0.000470,0
1916581,0.808391,0.312963,-0.449955,-0.030445,0.000045,0


In [18]:
columns_greater_than_10 = (df_false['pycbc_ifar'] > 10).sum()
print(f"Number of columns in 'ifar_false' greater than 10: {columns_greater_than_10}")

Number of columns in 'ifar_false' greater than 10: 358


In [19]:
columns_greater_than_10 = (df_true['pycbc_ifar'] > 10).sum()
print(f"Number of columns in 'ifar_true' greater than 10: {columns_greater_than_10}")

Number of columns in 'ifar_true' greater than 10: 63312


In [20]:
df = pd.concat([df_true, df_false], axis=0)
df

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
0,1.692460,0.501600,0.000000,-0.037187,0.007150,NaN
1,6.421466,0.763863,0.000000,0.021298,0.038439,NaN
2,10.481894,1.045793,0.000000,0.012784,-0.030501,NaN
3,8.906005,0.811978,0.000000,-0.000504,-0.004068,NaN
4,7.129897,0.808812,0.000000,-0.001127,-0.003302,NaN
...,...,...,...,...,...,...
1916578,0.864843,0.247605,0.000010,-0.416889,0.025385,0.0
1916579,0.501223,0.202115,0.000011,-0.743296,-0.011618,0.0
1916580,0.956881,0.213006,0.000470,-0.813010,0.049815,0.0
1916581,0.808391,0.312963,0.000045,-0.449955,-0.030445,0.0


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# Data exploration
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3833166 entries, 0 to 1916582
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   mass1       float64
 1   mass2       float64
 2   pycbc_ifar  float64
 3   spin1z      float64
 4   spin2z      float64
 5   label       float64
dtypes: float64(6)
memory usage: 204.7 MB


In [23]:
df.describe()

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
count,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,3.833166e+06,1916583.0
mean,3.471293e+00,4.917871e-01,3.175995e+02,-7.708433e-02,3.945316e-03,0.0
std,3.106359e+00,2.840450e-01,3.890053e+03,3.816217e-01,1.290550e-01,0.0
min,1.958331e-01,1.900007e-01,0.000000e+00,-9.000000e-01,-8.999972e-01,0.0
25%,9.126695e-01,2.351717e-01,0.000000e+00,-1.657461e-01,-3.013016e-02,0.0
50%,2.096459e+00,3.842256e-01,1.548821e-06,-6.712440e-03,1.399800e-03,0.0
75%,5.674761e+00,7.272952e-01,4.354362e-05,3.252966e-02,3.906670e-02,0.0
max,1.099997e+01,1.100000e+00,1.427988e+05,9.000000e-01,8.999999e-01,0.0


# Model with all data

In [24]:
X = df[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df['pycbc_ifar']

In [25]:
X

,mass1,mass2,spin1z,spin2z
0,1.692460,0.501600,-0.037187,0.007150
1,6.421466,0.763863,0.021298,0.038439
2,10.481894,1.045793,0.012784,-0.030501
3,8.906005,0.811978,-0.000504,-0.004068
4,7.129897,0.808812,-0.001127,-0.003302
...,...,...,...,...
1916578,0.864843,0.247605,-0.416889,0.025385
1916579,0.501223,0.202115,-0.743296,-0.011618
1916580,0.956881,0.213006,-0.813010,0.049815
1916581,0.808391,0.312963,-0.449955,-0.030445


In [26]:
y

0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
             ...   
1916578    0.000010
1916579    0.000011
1916580    0.000470
1916581    0.000045
1916582    0.000006
Name: pycbc_ifar, Length: 3833166, dtype: float64

In [27]:
def train_random_forest(X, y, n_estimators, ifar_threshold=10):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Report the current n_estimators
    print(f"Entrenando modelo con n_estimators={n_estimators}")

    # Create and train the model
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Calculate the custom prediction
    prediction_value = (y_pred > ifar_threshold).sum() / len(y_test) * 100
    
    # Show results with specific comments from n_estimators
    print(f"len(y_test) con n_estimators={n_estimators}: {len(y_test)}")
    print(f"(y_test > 10).sum() con n_estimators={n_estimators}: {(y_test > 10).sum()}")
    print(f"(y_pred > 10).sum() con n_estimators={n_estimators}: {(y_pred > 10).sum()}")
    print(f"predicción con n_estimators={n_estimators}: {prediction_value:.2f}%")
    print(f"MSE con n_estimators={n_estimators}: {mse:.4f}")
    print(f"RMSE con n_estimators={n_estimators}: {rmse:.4f}")
    print(f"MAE con n_estimators={n_estimators}: {mae:.4f}")
    print(f"MAPE con n_estimators={n_estimators}: {mape:.4f}")
    print(f"R^2 con n_estimators={n_estimators}: {r2:.4f}")
    print("-" * 40)

    # Return the model and metrics if necessary
    return model, mse, rmse, mae, mape, r2, prediction_value


In [28]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 766634
(y_test > 10).sum() con n_estimators=50: 12800
(y_pred > 10).sum() con n_estimators=50: 268668
predicción con n_estimators=50: 35.05%
MSE con n_estimators=50: 15594626.0026
RMSE con n_estimators=50: 3949.0032
MAE con n_estimators=50: 649.9769
MAPE con n_estimators=50: 1516028462232573440.0000
R^2 con n_estimators=50: -0.0344
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 15594626.002624659,
 3949.0031656893693,
 649.9769276330059,
 1.5160284622325734e+18,
 -0.03436341477444271,
 35.045145401847556)

# Model with df_true only

In [29]:
X = df_true[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_true['pycbc_ifar']

In [30]:
X

,mass1,mass2,spin1z,spin2z
0,1.692460,0.501600,-0.037187,0.007150
1,6.421466,0.763863,0.021298,0.038439
2,10.481894,1.045793,0.012784,-0.030501
3,8.906005,0.811978,-0.000504,-0.004068
4,7.129897,0.808812,-0.001127,-0.003302
...,...,...,...,...
1916578,5.784159,0.559250,-0.038265,-0.032295
1916579,8.621294,0.868851,0.032837,-0.010397
1916580,10.560706,0.717379,-0.029833,-0.031378
1916581,5.727628,0.886952,-0.024699,0.007844


In [31]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 383317
(y_test > 10).sum() con n_estimators=50: 12707
(y_pred > 10).sum() con n_estimators=50: 268856
predicción con n_estimators=50: 70.14%
MSE con n_estimators=50: 31154052.9721
RMSE con n_estimators=50: 5581.5816
MAE con n_estimators=50: 1294.5186
MAPE con n_estimators=50: 3061205483204955136.0000
R^2 con n_estimators=50: -0.0385
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 31154052.972071577,
 5581.58158339297,
 1294.5185676455058,
 3.061205483204955e+18,
 -0.03848417581909569,
 70.13933637172366)

# Model with df_true['ifar']>10 only

In [32]:
df_true = df_true[df_true['pycbc_ifar'] > 10]
df_true

,mass1,mass2,pycbc_ifar,spin1z,spin2z
17,10.044697,0.771064,10174.474264,0.003799,-0.034952
39,9.871082,0.608848,10174.474264,0.003390,0.002160
45,6.012620,0.290126,10174.474264,0.022516,0.025649
52,1.566219,0.522978,10174.474264,0.032162,-0.004396
59,5.331335,0.458099,10174.474264,-0.004934,0.009750
...,...,...,...,...,...
1903266,6.626040,0.982731,69.997932,-0.030900,-0.025199
1903269,10.661847,0.462810,25146.646963,0.011288,0.066585
1903305,2.810979,0.813847,54.111754,0.012777,-0.049325
1903359,8.898606,0.641294,25146.646963,-0.067872,-0.015317


In [33]:
X = df_true[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_true['pycbc_ifar']

In [34]:
X

,mass1,mass2,spin1z,spin2z
17,10.044697,0.771064,0.003799,-0.034952
39,9.871082,0.608848,0.003390,0.002160
45,6.012620,0.290126,0.022516,0.025649
52,1.566219,0.522978,0.032162,-0.004396
59,5.331335,0.458099,-0.004934,0.009750
...,...,...,...,...
1903266,6.626040,0.982731,-0.030900,-0.025199
1903269,10.661847,0.462810,0.011288,0.066585
1903305,2.810979,0.813847,0.012777,-0.049325
1903359,8.898606,0.641294,-0.067872,-0.015317


In [35]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 12663
(y_test > 10).sum() con n_estimators=50: 12663
(y_pred > 10).sum() con n_estimators=50: 12663
predicción con n_estimators=50: 100.00%
MSE con n_estimators=50: 588349382.1518
RMSE con n_estimators=50: 24255.9144
MAE con n_estimators=50: 18053.5588
MAPE con n_estimators=50: 73.4883
R^2 con n_estimators=50: -0.0630
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 588349382.1517744,
 24255.91437467931,
 18053.558778832285,
 73.48825671158474,
 -0.06296815505424469,
 100.0)

# Model with df_true['ifar']>10 and  df_false['ifar']<10 only

In [36]:
df_false = df_false[df_false['pycbc_ifar'] < 10]

df_false = df_false.sample(n=63312, random_state=42)

df_false

,mass1,mass2,spin1z,spin2z,pycbc_ifar,label
454132,1.934271,0.227301,-0.682905,-0.092424,0.000016,0
1093296,0.918194,0.473268,-0.510612,-0.048572,0.000343,0
1818994,0.607539,0.207085,-0.874255,-0.073717,0.000007,0
1263156,1.993887,0.621623,0.653651,0.050412,0.000007,0
1675300,0.566373,0.207698,0.667466,0.082528,0.000187,0
...,...,...,...,...,...,...
1669704,1.160685,0.497299,-0.426851,-0.076927,0.000017,0
261168,1.915233,0.556279,-0.239111,-0.718426,0.000351,0
458780,3.017803,0.351750,-0.843459,0.021864,0.000001,0
696421,1.536451,0.350264,0.304015,0.088003,0.000033,0


In [37]:
df = pd.concat([df_true, df_false], axis=0)
df

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
17,10.044697,0.771064,10174.474264,0.003799,-0.034952,NaN
39,9.871082,0.608848,10174.474264,0.003390,0.002160,NaN
45,6.012620,0.290126,10174.474264,0.022516,0.025649,NaN
52,1.566219,0.522978,10174.474264,0.032162,-0.004396,NaN
59,5.331335,0.458099,10174.474264,-0.004934,0.009750,NaN
...,...,...,...,...,...,...
1669704,1.160685,0.497299,0.000017,-0.426851,-0.076927,0.0
261168,1.915233,0.556279,0.000351,-0.239111,-0.718426,0.0
458780,3.017803,0.351750,0.000001,-0.843459,0.021864,0.0
696421,1.536451,0.350264,0.000033,0.304015,0.088003,0.0


In [38]:
X = df[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df['pycbc_ifar']

In [39]:
train_random_forest(X, y, 50, ifar_threshold=10)

Entrenando modelo con n_estimators=50
len(y_test) con n_estimators=50: 25325
(y_test > 10).sum() con n_estimators=50: 12733
(y_pred > 10).sum() con n_estimators=50: 13301
predicción con n_estimators=50: 52.52%
MSE con n_estimators=50: 295732894.7203
RMSE con n_estimators=50: 17196.8862
MAE con n_estimators=50: 9121.0291
MAPE con n_estimators=50: 9379022.6821
R^2 con n_estimators=50: 0.1862
----------------------------------------


(RandomForestRegressor(n_estimators=50, random_state=42),
 295732894.7203301,
 17196.886192573646,
 9121.029121127516,
 9379022.682093922,
 0.1861527103220132,
 52.521224086870674)